In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import io
import re
import json
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from math import ceil
import zipfile
import os


from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import google.cloud.storage

#Set up the azure storage
CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=stbusinesses;AccountKey=3Rytxhc0GguaI8g7OTZZc9ZWD1G2nzwfH5Ubuws2Pm6UAL78inYuJ5OZazCwpwkeTaXD4a5mS9R0+AStkP01QA==;EndpointSuffix=core.windows.net"
CONTAINER_NAME = "stbusinesses"

In [ ]:
def download_file(url):
    response = requests.get(url)
    return io.BytesIO(response.content)

In [ ]:
def upload_to_azure(data, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)
    container_client = blob_service_client.get_container_client(CONTAINER_NAME)
    blob_client = container_client.get_blob_client(blob_name)
    blob_client.upload_blob(data.getvalue(), overwrite=True)

Loan data: 

Go to https://data.sba.gov/dataset/ppp-foia 

13 CSV files on this specific page that need to be uploaded 

Example link of file: https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/738e639c-1fbf-4e16-beb0-a223831011e8/download/public_150k_plus_230930.csv 

In [ ]:
# Function to process PPP loan data
def process_ppp_loan_data():
    #
    base_url = "https://data.sba.gov/dataset/ppp-foia"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the link to each CSV file
    for link in soup.find_all('a', href=True):
        if link['href'].endswith('.csv'):
            file_url = link['href']
            file_name = file_url.split('/')[-1]
            print(f"Processing {file_name}")
            file_content = download_file(file_url)
            
            # Process the CSV file if needed
            df = pd.read_csv(file_content)
            # Perform any necessary transformations on df
            print(f"Number of rows in {file_name}: {len(df)}")
            
            # Upload to Azure
            output = io.StringIO()
            df.to_csv(output, index=False)
            output.seek(0)
            upload_to_azure(output, file_name)
            print(f"{file_name} uploaded to Azure successfully")
            


NAICS codes: 

Go to https://www.census.gov/naics/?48967 

Then, look for the most recent year's “NAICS Descriptions” XLSX file. In this case, it is “2022 NAICS Descriptions” and the link to the file is: 

https://www.census.gov/naics/2022NAICS/2022_NAICS_Descriptions.xlsx 

In [ ]:
def process_naics_data():
    base_url = "https://www.census.gov/naics/?48967"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the link to the latest NAICS Description XLXS file
    for link in soup.find_all('a', href=True):
        print(link['href'])



GDP data: 

You should then download a Zip file: 
https://apps.bea.gov/regional/zip/CAGDP1.zip 

From this Zip file, we only need the CSV that reads
“CAGDP1__ALL_AREAS”

In [ ]:
# Function to process GDP data
def process_gdp_data():
    pass

# Main 